In [1]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
import requests
import time
from shapely.geometry import box

# --- 1. Baixar o polígono do arthur alvim ---
bairro = ox.geocode_to_gdf("São Mateus, São Paulo")

# --- 2. Criar a grade (grid) sobre o bairro ---
def criar_grid(bairro, tamanho=0.005):  # Aproximadamente 200m
    minx, miny, maxx, maxy = bairro.total_bounds
    grid = []
    x = minx
    while x < maxx:
        y = miny
        while y < maxy:
            grid.append(box(x, y, x + tamanho, y + tamanho))
            y += tamanho
        x += tamanho
    grid_gdf = gpd.GeoDataFrame(geometry=grid, crs=bairro.crs)
    grid_gdf = gpd.overlay(grid_gdf, bairro, how='intersection')
    return grid_gdf

grid = criar_grid(bairro)


# --- 3. Corrigir projeção para metros, calcular centroide ---
grid = grid.to_crs(epsg=3857)  # Web Mercator
grid["centroide"] = grid.geometry.centroid

# --- 4. Criar colunas separadas de latitude/longitude
centroides = grid["centroide"].to_crs(epsg=4326)  # voltar para lat/lon
grid["centroide_lon"] = centroides.geometry.x
grid["centroide_lat"] = centroides.geometry.y

# --- 6. Definir hospital e API ---
API_KEY = "5b3ce3597851110001cf624824d680a2a89c4dc48933794aaca7b81d"  # Troque aqui pela sua chave
hospital_coords = [-46.4867109, -23.5947358]


def calcular_distancia_km(origem, destino, api_key):
    url = "https://api.openrouteservice.org/v2/directions/driving-car/json"
    headers = {
        "Authorization": api_key,
        "Content-Type": "application/json"
    }
    body = {
        "coordinates": [origem, destino]
    }
    try:
        response = requests.post(url, json=body, headers=headers)
        if response.status_code == 200:
            dados = response.json()
            distancia_metros = dados['routes'][0]['summary']['distance']
            distancia_km = distancia_metros / 1000
            return round(distancia_km, 2)
        else:
            print("Erro na API:", response.text)
            return None
    except Exception as e:
        print(f"Erro: {e}")
        return None

# --- 7. Calcular a distância para cada grid ---
distancias_km = []
for i, row in grid.iterrows():
    origem = [row["centroide_lon"], row["centroide_lat"]]
    distancia = calcular_distancia_km(origem, hospital_coords, API_KEY)
    distancias_km.append(distancia)
    time.sleep(1)  # evita limite da API gratuita

grid["distancia_km"] = distancias_km
grid.to_csv(f"estimativa_sao_mateus.csv", index=False)

In [2]:
grid

,bbox_west,bbox_south,bbox_east,bbox_north,place_id,osm_type,osm_id,lat,lon,class,...,place_rank,importance,addresstype,name,display_name,geometry,centroide,centroide_lon,centroide_lat,distancia_km
0,-46.506853,-23.624051,-46.454509,-23.575862,8004197,relation,3049453,-23.598299,-46.481705,boundary,...,18,0.390463,suburb,São Mateus,"São Mateus, São Paulo, Região Imediata de São ...","POLYGON ((-5176562.586 -2704018.89, -5176562.5...",POINT (-5176681.792 -2704105.294),-46.502924,-23.594762,2.31
1,-46.506853,-23.624051,-46.454509,-23.575862,8004197,relation,3049453,-23.598299,-46.481705,boundary,...,18,0.390463,suburb,São Mateus,"São Mateus, São Paulo, Região Imediata de São ...","POLYGON ((-5176562.586 -2703411.531, -5176562....",POINT (-5176835.677 -2703711.398),-46.504306,-23.591520,2.26
2,-46.506853,-23.624051,-46.454509,-23.575862,8004197,relation,3049453,-23.598299,-46.481705,boundary,...,18,0.390463,suburb,São Mateus,"São Mateus, São Paulo, Região Imediata de São ...","POLYGON ((-5176562.586 -2702804.194, -5176562....",POINT (-5176759.599 -2703134.547),-46.503623,-23.586771,2.89
3,-46.506853,-23.624051,-46.454509,-23.575862,8004197,relation,3049453,-23.598299,-46.481705,boundary,...,18,0.390463,suburb,São Mateus,"São Mateus, São Paulo, Região Imediata de São ...","MULTIPOLYGON (((-5176562.586 -2702804.194, -51...",POINT (-5176665.419 -2702627.328),-46.502777,-23.582595,2.68
4,-46.506853,-23.624051,-46.454509,-23.575862,8004197,relation,3049453,-23.598299,-46.481705,boundary,...,18,0.390463,suburb,São Mateus,"São Mateus, São Paulo, Região Imediata de São ...","POLYGON ((-5176562.586 -2702196.881, -5176571....",POINT (-5176588.341 -2701978.957),-46.502084,-23.577257,3.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,-46.506853,-23.624051,-46.454509,-23.575862,8004197,relation,3049453,-23.598299,-46.481705,boundary,...,18,0.390463,suburb,São Mateus,"São Mateus, São Paulo, Região Imediata de São ...","POLYGON ((-5172109.806 -2705233.678, -5172109....",POINT (-5171905.662 -2704961.864),-46.460019,-23.601813,3.62
65,-46.506853,-23.624051,-46.454509,-23.575862,8004197,relation,3049453,-23.598299,-46.481705,boundary,...,18,0.390463,suburb,São Mateus,"São Mateus, São Paulo, Região Imediata de São ...","POLYGON ((-5172109.806 -2704626.273, -5172109....",POINT (-5172046.137 -2704529.761),-46.461281,-23.598256,3.07
66,-46.506853,-23.624051,-46.454509,-23.575862,8004197,relation,3049453,-23.598299,-46.481705,boundary,...,18,0.390463,suburb,São Mateus,"São Mateus, São Paulo, Região Imediata de São ...","POLYGON ((-5171553.209 -2706448.56, -5171523.1...",POINT (-5171538.09 -2706463.801),-46.456717,-23.614176,4.62
67,-46.506853,-23.624051,-46.454509,-23.575862,8004197,relation,3049453,-23.598299,-46.481705,boundary,...,18,0.390463,suburb,São Mateus,"São Mateus, São Paulo, Região Imediata de São ...","POLYGON ((-5171553.209 -2706448.56, -5171553.2...",POINT (-5171458.606 -2706071.942),-46.456003,-23.610951,4.86


In [3]:
%pip install folium
import folium
from shapely.geometry import mapping

# Converter o bairro para EPSG:4326 (lat/lon) para exibição no mapa
bairro_wgs84 = bairro.to_crs(epsg=4326)
grid_wgs84 = grid.to_crs(epsg=4326)

# Criar mapa centralizado no bairro
centroide_bairro = bairro_wgs84.geometry.centroid.iloc[0]
m = folium.Map(location=[centroide_bairro.y, centroide_bairro.x], zoom_start=14)

# Adicionar polígono do bairro
folium.GeoJson(bairro_wgs84.geometry.__geo_interface__,
               name="Bairro - Tatuape",
               style_function=lambda x: {"color": "black", "fillOpacity": 0.1}
).add_to(m)

# Adicionar os grids
for _, row in grid_wgs84.iterrows():
    folium.GeoJson(mapping(row.geometry),
                   style_function=lambda x: {"color": "blue", "weight": 1, "fillOpacity": 0}
    ).add_to(m)

# Adicionar ponto do hospital
, 
folium.Marker(
    location=[-23.5947358, -46.4867109] ,
    popup="Hospital São Mateus",
    icon=folium.Icon(color='red', icon='plus-sign')
).add_to(m)

# Mostrar mapa
m

# [-46.5315852, -23.5605378] 


[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


C:\Users\joth1\AppData\Local\Temp\ipykernel_17652\2673571646.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroide_bairro = bairro_wgs84.geometry.centroid.iloc[0]
